In [10]:
import pandas as pd
import numpy as np
import json

In [11]:
with open("hp_ebs.json", "r") as f:
    result = json.load(f)


dfs_ebs = {}
for key in ["components", "connections"]:
    dfs_ebs[key] = pd.DataFrame.from_dict(result[key], orient="index").sort_index()

In [12]:
with open("hp_tespy.json", "r") as f:
    result = json.load(f)


dfs_tespy = {}
for key in ["components", "connections"]:
    dfs_tespy[key] = pd.DataFrame.from_dict(result[key], orient="index").sort_index()

In [13]:
dfs_ebs["connections"][["e_PH"]] = dfs_ebs["connections"][["e_PH"]].round(6)
dfs_tespy["connections"][["e_PH"]] = dfs_tespy["connections"][["e_PH"]].round(6)

In [14]:
columns = ["m", "p", "T", "e_T", "e_M"]

overlapping_index = list(set(dfs_tespy[key].index.tolist()) & set(dfs_ebs[key].index.tolist()))
diff_to_tespy = (
    (
        dfs_ebs[key].loc[overlapping_index, columns]
        - dfs_tespy[key].loc[overlapping_index, columns]
    ) / dfs_tespy[key].loc[overlapping_index, columns]
).abs().replace(np.inf, np.nan)
diff_to_ebs = (
    (
        dfs_ebs[key].loc[overlapping_index, columns]
        - dfs_tespy[key].loc[overlapping_index, columns]
    ).abs() / dfs_tespy[key].loc[overlapping_index, columns]
).abs().replace(np.inf, np.nan)

In [15]:
diff_to_ebs

,m,p,T,e_T,e_M
23,0.000000,0.000000e+00,8.249821e-11,4.561179e-07,4.723624e-04
11,0.000002,0.000000e+00,4.416582e-14,NaN,NaN
30,0.000079,0.000000e+00,2.481835e-09,1.481377e-04,0.000000e+00
21,0.000000,0.000000e+00,5.584974e-10,1.440545e-06,4.723624e-04
13,0.000002,0.000000e+00,3.415439e-10,2.660484e-04,NaN
12,0.000002,9.975535e-10,1.134165e-05,2.645430e-03,2.433095e-04
32,0.000079,0.000000e+00,5.074101e-09,1.274528e-04,2.583452e-07
31,0.000079,0.000000e+00,2.107598e-09,1.481416e-04,0.000000e+00
34,0.000079,0.000000e+00,3.992981e-05,6.342950e-05,0.000000e+00
33,0.000079,1.459999e-10,1.154451e-05,2.720777e-03,2.157887e-07


In [16]:
diff_to_ebs[diff_to_ebs > 1e-3].dropna(how="all")

,m,p,T,e_T,e_M
12,NaN,NaN,NaN,0.002645,NaN
33,NaN,NaN,NaN,0.002721,NaN


In [17]:
# Load the CSV files
tespy_results = pd.read_csv("hp_components_tespy.csv", index_col=0)
ebsilon_results = pd.read_csv("hp_components_ebsilon.csv", index_col=0)

# Step 1: Merge DataFrames on Component name
merged_df = pd.merge(ebsilon_results, tespy_results, on="Component", suffixes=("_Ebsilon", "_Tespy"))

# Step 2: Compute Differences
columns_to_compare = ["E_F [kW]", "E_P [kW]", "E_D [kW]", "E_L [kW]", "ε [%]", "y [%]", "y* [%]"]
for col in columns_to_compare:
    merged_df[f"Diff_{col}"] = merged_df[f"{col}_Tespy"] - merged_df[f"{col}_Ebsilon"]

# Step 3: Select only the relevant columns
diff_columns = [col for col in merged_df.columns if col.startswith("Diff_E_D") or col.endswith("E_D [kW]_Ebsilon") or col.endswith("E_D [kW]_Tespy")]
diff_df = merged_df[["Component"] + diff_columns]
diff_df["Relative error [%]"] = diff_df["Diff_E_D [kW]"] / diff_df["E_D [kW]_Tespy"] * 100

# Display the filtered DataFrame
diff_df

C:\Users\sergiotomasinelli\AppData\Local\Temp\ipykernel_13436\63166597.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  diff_df["Relative error [%]"] = diff_df["Diff_E_D [kW]"] / diff_df["E_D [kW]_Tespy"] * 100


,Component,E_D [kW]_Ebsilon,E_D [kW]_Tespy,Diff_E_D [kW],Relative error [%]
0,COMP,125.814542,125.819130,0.004587,0.003646
1,COND,91.493965,91.482067,-0.011897,-0.013005
2,EVA,647.197052,647.061438,-0.135615,-0.020959
3,FAN,104.880549,104.855562,-0.024986,-0.023829
4,PUMP,0.010569,0.010547,-0.000022,-0.212093
5,VAL,144.574924,144.605102,0.030178,0.020869
6,TOT,1138.332290,1138.192444,-0.139847,-0.012287
